### MAIN: Data Cleaner
#### Choose clean non-pooled data only

In [5]:
import json
import pandas as pd

# Function to convert time in seconds to floating-point seconds
def convert_to_seconds(time, unit):
    if unit == "seconds":
        return float(time)
    elif unit == "minutes":
        return float(time) * 60  # Convert minutes to seconds

# Step 1: Load the data from "new.json" with multiple JSON objects separated by new lines
with open("new.json", "r") as json_file:
    new_data = [json.loads(line) for line in json_file]  # Read each line as a JSON object

# Step 2: Load the data from "pooling.csv"
pooling_data = pd.read_csv("anon_pooling_jan_24_amman.csv")

# Step 3: Filter pooling_data to only include records with the 'day' attribute set to '2024-01-31'
filtered_pooling_data = pooling_data[pooling_data['day'] == '2024-01-31']

output_file = "pooling_by_day.csv"
filtered_pooling_data.to_csv(output_file, index=False)
# with open("pooling_by_day.csv", "w") as output_file:
    # json.dump(filtered_pooling_data, output_file)


Matching process completed. Check 'clean_trips.json' for results.


In [9]:
import pandas as pd

input_file = "pooling_by_day.csv"
filtered_pooling_data = pd.read_csv(input_file)

# Step 1: Load the data from "new.json" with multiple JSON objects separated by new lines
with open("new.json", "r") as json_file:
    new_data = [json.loads(line) for line in json_file]  # Read each line as a JSON object

output_file = "clean_trips.json"

# Step 3: Initialize an empty list to store matched records
# matched_records = []

with open(output_file_path, 'w', encoding='utf-8') as output_file:
# Step 4: Iterate over the data from "new.json"
    for entry in new_data:
        entry_time = entry["time"]
        entry_trip_id = entry["trip_id"]
        entry_driver_id = entry["driverID"]

        # Step 5: Search for a matching entry in "pooling.csv"
        for _, row in filtered_pooling_data.iterrows():
            pool_time = float(row["captain_engagement_time"] * 60)
            pool_trip_id = row["booking_id"]
            pool_driver_id = row["captain_id"]

            # Step 6: Check for trip_id, driver_id, and time margin of 5 seconds
            if (entry_trip_id == pool_trip_id and
                entry_driver_id == pool_driver_id and
                abs(pool_time - entry_time) <= 5):
                # If match found, add to matched_records
                # matched_records.append(entry)
                # with open(output_file_path, 'w', encoding='utf-8') as output_file:
                #     output_file.write(entry)
                json.dump(entry, output_file)
                output_file.write("\n")
                break

# Step 7: Write the matched records to "clean_trips.json"
# with open("clean_trips.json", "w") as output_file:
#     json.dump(matched_records, output_file)

print("Matching process completed. Check 'clean_trips.json' for results.")

Matching process completed. Check 'clean_trips.json' for results.


Get Unclean trips

not this version


In [11]:
import json
import os

# Function to read a JSON file and return its content
def read_json(filename):
   data=[]
   with open(filename, 'r') as file:
        # data = json.load(file)
        for i in file:
         data.append(json.loads(i))   
   return data

# Function to write a JSON file
def write_json(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

# Read data from the two JSON files
new_trips = read_json('new_uncleaned1.json')  # Reading new data
clean_trips = read_json('clean_trips.json')  # Reading existing clean trips


# Set of booking IDs in clean_trips.json for faster lookup
clean_booking_ids = {trip['trip_id'] for trip in clean_trips}

# List to store unmatched trips
unmatched_trips = []

# Check each entry in new_trips to see if its booking_id is in clean_booking_ids
for trip in new_trips:
    booking_id = trip.get('trip_id')
    if booking_id not in clean_booking_ids:
        # If the booking ID is not in clean trips, add it to unmatched_trips
        unmatched_trips.append(trip)

# If there are unmatched trips, write them to a new JSON file
if unmatched_trips:
    write_json(unmatched_trips, 'unmatched_trips.json')  # Write to a new file
    print("Unmatched trips found and saved to 'unmatched_trips.json'.")
else:
    print("No unmatched trips found.")


Unmatched trips found and saved to 'unmatched_trips.json'.


In [12]:
import json

def find_unmatched_trips(new_file_path, clean_file_path, output_file_path):
    # Read data from the two JSON files
    new_trips = []
    with open(new_file_path, 'r') as new_file:
        for line in new_file:
            new_trips.append(json.loads(line))

    clean_trips = []
    with open(clean_file_path, 'r') as clean_file:
        for line in clean_file:
            clean_trips.append(json.loads(line))

    # Extract trip IDs from clean_trips
    clean_trip_ids = {trip['trip_id'] for trip in clean_trips}

    # Find unmatched trips
    unmatched_trips = [trip for trip in new_trips if trip['trip_id'] not in clean_trip_ids]

    # Write unmatched trips to a new JSON file
    with open(output_file_path, 'w') as output_file:
        for trip in unmatched_trips:
            json.dump(trip, output_file)
            output_file.write('\n')

# Example usage:
new_file_path = 'new_uncleaned1.json'
clean_file_path = 'clean_trips.json'
output_file_path = 'unmatched_trips2.json'

find_unmatched_trips(new_file_path, clean_file_path, output_file_path)


